# Redundant

In [1]:
using Random, Distributions

In [110]:
########################### Redundant!!!!!!
# time_st1 = Vector{Float64}()
# time_st2 = Vector{Float64}()
# time_from_st1 = Vector{Float64}()
# time_from_st2 = Vector{Float64}()
# n_st1_vect = Vector{Int64}()
# n_st2_vect = Vector{Int64}()

# draw_st1 = rand(Exponential(10))
# draw_st2 = rand(Exponential(10))



# n_st1 = 5
# n_st2 = 5
# n_total = n_st1 + n_st2
# n_travel = 0
# travel_f_1 = 100 #problem - Interrem ingen rejser. ingen er op vej nogen steder hen
# travel_f_2 = 100
# shit_1 = 0 #Nogen som ankommer og går uden at blive serviceret. 
# shit_2 = 0

# push!(time_st1,draw_st1) #Smider de første værdier ind i vektoren. 
# push!(time_st2,draw_st2)
# push!(time_from_st1,travel_f_1)
# push!(time_from_st2,travel_f_2)
# push!(n_st1_vect,n_st1)
# push!(n_st2_vect,n_st2)


# # antager at hvis der ikke er en bil løber folk væk igen og der gåe en ny tid før studerende ankommer


# for i in 1:10 #placeholder for whileloop. 
#     # Crude as shit!
#     if last(time_st1) < last(time_st2) && last(time_st1) < last(time_from_st1) && last(time_st1) < last(time_from_st2)
        
#         if n_st1 >= 1 && n_st1 <= n_travel+n_st1+n_st2 
#             n_st1 -= 1
#             push!(n_st1_vect,n_st1)
#             travel_f_1 = rand(Exponential(10))
#             n_travel +=1
#             push!(time_from_st1,travel_f_1+last(time_st1))
        
#         else
#             shit_1 += 1
#         end

#         draw_st1 = rand(Exponential(10))
#         push!(time_st1,last(time_st1)+draw_st1)

#     elseif last(time_st2) < last(time_st1) && last(time_st2) < last(time_from_st1) && last(time_st2) < last(time_from_st2)
        
#         if n_st2 >= 1 && n_st2 <= n_travel+n_st1+n_st2
#             n_st2 -= 1
#             push!(n_st2_vect,n_st2)
#             travel_f_2 = rand(Exponential(10))
#             n_travel += 1
#             push!(time_from_st2,travel_f_2+last(time_st2))
            
#         else
#             shit_2 += 1
#         end

#         draw_st2 = rand(Exponential(10))
#         push!(time_st2,last(time_st2)+draw_st2)

#     elseif last(time_st1) > last(time_from_st1) && last(time_st2) > last(time_from_st1) && last(time_from_st1) < last(time_from_st2) && n_travel+n_st1+n_st2 == n_total
#         n_st2 += 1
#         push!(n_st2_vect,n_st2+last(n_st2_vect))
#         n_travel -= 1
        
#         push!(time_from_st1,100)

#     elseif last(time_st1) > last(time_from_st2) && last(time_st2) > last(time_from_st2) && last(time_from_st1) > last(time_from_st2) && n_travel+n_st1+n_st2 == n_total
#         n_st1 += 1
#         push!(n_st1_vect,n_st1+last(n_st1_vect))
#         n_travel -= 1
        
#         push!(time_from_st2,100)
#     else
#         println('Something Wrong.')
#     end
# end


# println(time_st1,time_from_st1)


aj
na
lon
la
aj
lon
aj
aj
la
lon
[71.62357998333988, 63.093335020066434, 33.313651003967, 9.832390594385078][64.9515268496291, 100.0, 37.834199381928, 100.0, 18.21233596117552, 100.0]


# The Real code

In [2]:
using Random, Distributions, Plots

In [33]:
# Using tuples instead of PriorityQueue, because they can have multiple identical keys and the queue cannot.
# This is slightly slower but not by that much, not really a noticable difference in this case. 
# You have to get real up there in iterations before it really chugs behind.

mutable struct Events
    e::Array{Tuple{Float64,SubString},1}
end

struct Environment
    hours::Float64
    lambda::Float64  #arrival rate
    mu::Float64 #service rate
   
   #Add arrival rate per hour for both stations. Add travel-time rate for both stations.
    Environment() = new(
    2, 
    (1/6), 
    (1/12)
    )
end

mutable struct Stats
    n_cars_st1::Int64
    n_cars_st2::Int64
    unfilled_st1::Int64
    unfilled_st2::Int64
  
end

# Might not be just as fast as PriorityQueue but it is not that much behind
function insert_and_sort!(vect::Vector, x)
    (splice!(vect, searchsorted(vect,x), [x]); vect) 
end

insert_and_sort! (generic function with 1 method)

In [101]:
function arrivals_generator(event::Events,env::Environment)

    # Arrivals at station 1
    push!(event.e,(rand(Exponential(env.lambda)),"st1")) # First arrival
    while last(event.e)[1] <= env.hours
       push!(event.e,(rand(Exponential(env.lambda))+last(event.e)[1],"st1"))
    end

    # Arrivals at station 2 (repeat of station 1)
    push!(event.e,(rand(Exponential(env.lambda)),"st2"))
    while last(event.e)[1] <= env.hours
        push!(event.e,(rand(Exponential(env.lambda))+last(event.e)[1],"st2"))
    end


    # Sorting the arrivals so first time is 1 and so on
    sort!(event.e)

    return e
end

arrivals_generator (generic function with 2 methods)

In [ ]:
# Do this:
# stats.overview_usage = [stats.overview_usage; t state.queue_length state.server_busy]
# Real nice, steal that shit!

function arrival(station,event::Events,env::Enviroment,stat::Stats)
    # WHAT to do when travel time passes total time?? makes sense? ask teacher
    if station == "st1"
        # Student arrives at empty station. That is one unsatisfied costumer :(
        if stat.n_cars_st1 == 0
            stat.unfilled_st1 += 1
            popfirst!(event.e)
        else
            stat.n_cars_st1 -= 1
        
            insert_and_sort!(event.e,(rand(Exponential(env.mu))+event.e[1][1],"que_st2") )
            popfirst!(event.e)
        end
        
    elseif station == "st2"
        if stat.n_cars_st1 == 0
            stat.unfilled_st2 += 1
            popfirst!(event.e)
        else
            stat.n_cars_st2 -= 1
            
            insert_and_sort!(event.e,(rand(Exponential(env.mu))+event.e[1][1],"que_st1") )
            popfirst!(event.e)
        end

    else
        throw(ErrorException("Something went wrong in main()."))
    end

end 

function travel(station,e = e,env::Enviroment,stat::Stats)

    if station == "que_st1"
        stat.n_cars_st1 += 1
        popfirst!(event.e)
    elseif station == "que_st2"
        stat.n_cars_st1 += 1
        popfirst!(event.e)
    else 
        throw(ErrorException("Something went wrong in main()."))
    end

end


In [ ]:
function main()
    
end

### Testing

In [117]:
env = Environment()
arrivals_generator(env)

25-element Vector{Tuple{Float64, SubString}}:
 (0.18304872917366644, "st1")
 (0.32418133395301874, "st1")
 (0.42962937673558826, "st1")
 (0.8205059019465195, "st2")
 (0.8533169611400817, "st2")
 (0.9529054677557682, "st1")
 (0.9636302296602116, "st2")
 (1.0208796562297833, "st1")
 (1.1300790230053983, "st1")
 (1.1832036918799527, "st1")
 ⋮
 (1.6808449610470393, "st2")
 (1.6941841241859048, "st1")
 (1.7111897215440317, "st2")
 (1.7177486490598652, "st2")
 (1.8605707354780172, "st2")
 (1.8611077166784153, "st1")
 (1.9427148423650873, "st1")
 (1.9598780802176874, "st2")
 (1.9935731680409337, "st2")

In [27]:
v = Array{Tuple{Float64,String},1}([(1,"1"),(4,"2"),(223,"3")])
sort!(v)
insert_and_sort!(v, (2,"4"))
v

4-element Vector{Tuple{Float64, String}}:
 (1.0, "1")
 (2.0, "4")
 (4.0, "2")
 (223.0, "3")